## Chainstack Subgraphs: how to query Uniswap V2

<div>
    <img src="https://chainstack.com/wp-content/themes/chainstack/img/press-kit-logo-white-text.svg" width="300"/><br>
    <img src="https://images.ctfassets.net/oc3ca6rftwdu/3phk8Req8L5YvWpoQx7Jtv/0cf6f1125f0aef8929203894433e487e/Banner.png" width="300"/>
    <br><br>
</div>


This tutorial demonstrates how to query Chainstack Subgraphs, specifically the Uniswap V2 subgraph, using the `Subgrounds` library.

Before diving into the queries, ensure your environment is ready to work with the Subgrounds library. This involves installing necessary dependencies and configuring your Jupyter Notebook for an optimal experience.

## Install Required Packages

First, ensure you have the necessary Python packages installed. You can run the following command in a Jupyter Notebook cell:

In [2]:
%pip install subgrounds pandas

Note: you may need to restart the kernel to use updated packages.


## Importing Dependencies

Let's start by importing the necessary libraries in a Jupyter Notebook cell:

In [3]:
from subgrounds import Subgrounds
import pandas as pd

import warnings

# Suppress ResourceWarnings related to unclosed sockets
warnings.filterwarnings('ignore', category=ResourceWarning)

# Set column width
pd.set_option('display.max_colwidth', None)

## Connecting to the Subgraph

Now, we'll connect to the Uniswap V2 subgraph using the Subgrounds library. This involves initializing the Subgrounds object:

In [7]:
sg = Subgrounds(timeout=30)

subgraph = sg.load_subgraph('YOUR_CHAINSTACK_ENDPOINT_HERE')

## Get latest swaps

This query retrieves the latest swaps, sorted by timestamp in descending order. It includes details such as transaction ID, timestamp, token pair symbols, and swap amounts in USD.

In [9]:
swaps_query = subgraph.Query.swaps(
    first=5,
    orderBy='timestamp',
    orderDirection='desc'
)

sg.query_df(
    [
        swaps_query.transaction.id,
        swaps_query.transaction.timestamp,
        swaps_query.pair.token0.symbol,
        swaps_query.pair.token1.symbol,
        swaps_query.amountUSD
    ],
    columns=[
        'Tx',
        'Timestamp',
        'Token 0',
        'Token 1',
        'Amount USD'
    ]
)

,Tx,Timestamp,Token 0,Token 1,Amount USD
0,0xee0ad5ab4ceea8373e1eb10746b07d0659ed63f8d54a773f2f19f1cb81e97600,1723454651,WETH,TrumpX,1286.441675
1,0xde647eb7736b7a2d4d184dc725358a17ad435bf6a0a89b8e408333ad1e534677,1723454651,WETH,TrumpX,2522.158046
2,0xdc325833c0a179ecab1ab969319f342029d7aa3b3a419cbc3cf985efadf547df,1723454651,WETH,TrumpX,516.829455
3,0xc987dc9276ea0308acff596db266264f0d6a52fa235b3df74fd6a806dee9bf9a,1723454651,KEN,WETH,1139.422222
4,0xc8bfd8b6d6011fb6fac7b858b4ce3f9d09a864d42b83c67202843e352cd799b4,1723454651,WETH,TrumpX,551.694181


## Get token price

This query fetches the price of a specific token in ETH, along with its symbol and name, by querying the `token` entity. It also retrieves the current ETH price from the `bundles` entity.


In [11]:
token_id = '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'

token_price_query = subgraph.Query.token(id=token_id)

sg.query_df(
    [
        token_price_query.id,
        token_price_query.symbol,
        token_price_query.name,
        token_price_query.derivedETH,
        subgraph.Query.bundles.ethPrice
    ]
)

,token_id,token_symbol,token_name,token_derivedETH,bundles_ethPrice
0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,Wrapped BTC,22.595847,2582.694499


## Get hourly trading volume of a pair

This query retrieves hourly trading volumes for a specific token pair, filtered by `hourStartUnix_gte` to specify a time range.

In [12]:
pair_id = '0xbb2b8038a1640196fbe3e38816f3e67cba72d940'  # WBTC/WETH pair ID
start_time_unix = 1719788422  # Unix timestamp for the start time

hourly_volume_query = subgraph.Query.pairHourDatas(
    where={
        'pair_': {'id': pair_id},
        'hourStartUnix_gte': start_time_unix
    },
    orderBy='hourStartUnix',
    orderDirection='desc'
)

# Retrieve data as a DataFrame
sg.query_df(
    [
        hourly_volume_query.hourStartUnix,
        hourly_volume_query.pair.token0.symbol,
        hourly_volume_query.hourlyVolumeToken0,
        hourly_volume_query.pair.token1.symbol,
        hourly_volume_query.hourlyVolumeToken1,
        hourly_volume_query.hourlyVolumeUSD,
        hourly_volume_query.hourlyTxns
    ],
    columns=[
        'Hour Start Unix',
        'Token 0',
        'Volume Token 0',
        'Token 1',
        'Volume Token 1',
        'Volume USD',
        'Hourly Transactions'
    ]
)

,Hour Start Unix,Token 0,Volume Token 0,Token 1,Volume Token 1,Volume USD,Hourly Transactions
0,1723453200,WBTC,0.438844,WETH,9.907917,25617.924404,9
1,1723449600,WBTC,0.247127,WETH,5.620964,14543.256088,6
2,1723446000,WBTC,0.124859,WETH,2.851410,7297.849131,3
3,1723442400,WBTC,0.135465,WETH,3.102377,7921.728346,8
4,1723438800,WBTC,0.028342,WETH,0.650246,1658.293762,4
...,...,...,...,...,...,...,...
95,1723096800,WBTC,0.107323,WETH,2.524333,6113.183391,3
96,1723093200,WBTC,0.010747,WETH,0.252453,612.579273,1
97,1723089600,WBTC,0.000105,WETH,0.002473,6.001830,1
98,1723086000,WBTC,0.130864,WETH,3.069241,7524.344455,7


## Get daily trading volume of a pair

This query retrieves daily trading volumes for a specific token pair, sorted by date for the last 7 days.

In [13]:
pair_address = '0xbb2b8038a1640196fbe3e38816f3e67cba72d940'  # WBTC/WETH pair address
num_days = 7  # Number of days to retrieve

daily_volume_query = subgraph.Query.pairDayDatas(
    where={
        'pairAddress': pair_address
    },
    orderBy='date',
    orderDirection='desc',
    first=num_days
)

sg.query_df(
    [
        daily_volume_query.date,
        daily_volume_query.dailyVolumeUSD,
        daily_volume_query.dailyTxns,
        daily_volume_query.token0.symbol,
        daily_volume_query.dailyVolumeToken0,
        daily_volume_query.token1.symbol,
        daily_volume_query.dailyVolumeToken1
    ],
    columns=[
        'Date',
        'Daily Volume USD',
        'Daily Transactions',
        'Token 0',
        'Daily Volume Token 0',
        'Token 1',
        'Daily Volume Token 1'
    ]
)

,Date,Daily Volume USD,Daily Transactions,Token 0,Daily Volume Token 0,Token 1,Daily Volume Token 1
0,1723420800,97874.659313,51,WBTC,1.672988,WETH,38.189998
1,1723334400,155886.498371,129,WBTC,2.572445,WETH,59.114958
2,1723248000,96498.326477,123,WBTC,1.588761,WETH,36.971128
3,1723161600,223827.869640,119,WBTC,3.681795,WETH,84.689859
4,1723075200,603025.952276,146,WBTC,10.209026,WETH,237.681444
5,1722988800,364415.428807,174,WBTC,6.465059,WETH,150.085206
6,1722902400,238665.675119,118,WBTC,4.266771,WETH,95.505356


## Get recently created pools

This query retrieves the most recently created pools, sorted by `createdAtBlockNumber` in descending order.

In [14]:
num_pools = 10  # Number of pools to retrieve
order_by_block_number = 'createdAtBlockNumber'
order_direction_desc = 'desc'

recent_pools_query = subgraph.Query.pairs(
    first=num_pools,
    orderBy=order_by_block_number,
    orderDirection=order_direction_desc
)

sg.query_df(
    [
        recent_pools_query.id,
        recent_pools_query.createdAtTimestamp,
        recent_pools_query.createdAtBlockNumber,
        recent_pools_query.token0.symbol,
        recent_pools_query.token0.id,
        recent_pools_query.token1.symbol,
        recent_pools_query.token1.id,
        recent_pools_query.reserveUSD,
        recent_pools_query.reserveETH
    ],
    columns=[
        'Pair ID',
        'Created At Timestamp',
        'Created At Block Number',
        'Token 0 Symbol',
        'Token 0 ID',
        'Token 1 Symbol',
        'Token 1 ID',
        'Reserve USD',
        'Reserve ETH'
    ]
)

,Pair ID,Created At Timestamp,Created At Block Number,Token 0 Symbol,Token 0 ID,Token 1 Symbol,Token 1 ID,Reserve USD,Reserve ETH
0,0x8501b77ee1c62a80635c5bbde518b78d671eab80,1723457207,20511830,EHT,0x2394f65f523927906b43a4934d0cdad5364df1f5,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,25916.422938,10.000000
1,0x05c578a794742781b72d8451a5ff0eb6c98554c4,1723457135,20511824,BRATCAT,0xb58e2a870bc11a10c6109fe8a6eebff1e0836a24,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,96414.501195,37.202087
2,0xdc6dcb5ad109963da908bfd16be9cf6531f93212,1723457111,20511822,TEST,0xb8b7eb14965c7fb1a7efd018728c3bbfd303e1f2,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,77.810329,0.030023
3,0xcce3b9ce83e0820b92d2689433299e21c7788ffc,1723457075,20511819,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,TrumpX,0xdc014d1a58fd3a9b30556af5d336eec822bbe0fb,0.000000,0.000000
4,0x83b57006d2e1b76367e21f3818391792bc816c53,1723456811,20511798,$Trump0nX,0x0000000000f814b7d577a44ad7002b1776918ab4,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2730.387740,1.053529
5,0x0fc66e1c2979d6381c7e49a3fc19fa0e781935b7,1723456643,20511785,TrumpWithX,0x191956c51ae9034fe7266620bd3ac0f10f06d66c,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,59846.625577,23.091780
6,0xe4a26939aed5f940f9e28e4af40a027e79adaf87,1723456583,20511780,DOGS,0x6de40d8b68e86001dcff26c858759f17d6caed2f,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,284840.184872,109.907214
7,0xeff46fc540609109f7b6a89075e1c5a4ea3bf590,1723456175,20511746,ONE,0x764daf2351f243430711771f591508e0c45e2490,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,19904.912546,7.680424
8,0xc3087fd7e3c51458ed4c04da799f382957cab4f1,1723456055,20511736,brat,0x843449e93fed1f39182a6473b8c6c45f290733de,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,6126.007906,2.366676
9,0xe646e92cb7200c26f0630801179305f7adc5c273,1723455995,20511731,TrumpOnX,0x59a8e0ca167b55f8dfae4566535fc63e8863e315,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.000000,0.000000


## Get historical data on liquidity positions in a pair

This query retrieves historical snapshots of liquidity positions for a specific pair, filtered by the pair ID and sorted by `timestamp` in descending order.

In [16]:
pair_id = '0xbb2b8038a1640196fbe3e38816f3e67cba72d940'  # WBTC/WETH pair ID
order_by_timestamp = 'timestamp'
order_direction_desc = 'desc'
liquidity_threshold = 0  # Minimum liquidity token balance

historical_liquidity_positions_query = subgraph.Query.liquidityPositionSnapshots(
    orderBy=order_by_timestamp,
    orderDirection=order_direction_desc,
    where={
        'pair_': {'id': pair_id},
        'liquidityTokenBalance_gt': liquidity_threshold
    }
)

sg.query_df(
    [
        historical_liquidity_positions_query.block,
        historical_liquidity_positions_query.liquidityTokenBalance,
        historical_liquidity_positions_query.timestamp,
        historical_liquidity_positions_query.user.id,
        historical_liquidity_positions_query.liquidityTokenTotalSupply
    ],
    columns=[
        'Block',
        'Liquidity Token Balance',
        'Timestamp',
        'User ID',
        'Liquidity Token Total Supply'
    ]
)

,Block,Liquidity Token Balance,Timestamp,User ID,Liquidity Token Total Supply
0,20508816,1.116192e-07,1723420835,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004538
1,20501596,1.116192e-07,1723333703,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004548
2,20494464,1.116192e-07,1723247759,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004547
3,20487293,1.116192e-07,1723161323,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004539
4,20480231,1.116192e-07,1723076291,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004550
...,...,...,...,...,...
95,19994841,5.379624e-06,1717219235,0xc48189f6aadc9902be4475eeaaa4cbda729860af,0.003511
96,19992989,3.764028e-09,1717196867,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004530
97,19978698,3.764028e-09,1717024199,0x223bc79156cbb0a6d175ea6130cb382d01868df8,0.004530
98,19977949,1.218581e-05,1717015175,0xc6c11f32d3ccc3beaac68793bc3bfbe82838ca9f,0.003511


## Get Uniswap performance metrics

This query retrieves Uniswap's daily performance metrics, including daily volume and total liquidity, using the `uniswapDayDatas` entity.

In [17]:
num_days = 10  # Number of days to retrieve
order_by_date = 'date'
order_direction_desc = 'desc'

uniswap_performance_query = subgraph.Query.uniswapDayDatas(
    first=num_days,
    orderBy=order_by_date,
    orderDirection=order_direction_desc
)

sg.query_df(
    [
        uniswap_performance_query.date,
        uniswap_performance_query.dailyVolumeETH,
        uniswap_performance_query.dailyVolumeUSD,
        uniswap_performance_query.totalLiquidityETH,
        uniswap_performance_query.totalLiquidityUSD
    ],
    columns=[
        'Date',
        'Daily Volume ETH',
        'Daily Volume USD',
        'Total Liquidity ETH',
        'Total Liquidity USD'
    ]
)

,Date,Daily Volume ETH,Daily Volume USD,Total Liquidity ETH,Total Liquidity USD
0,1723420800,32220.146430,8.237414e+07,659455.281446,1.709240e+09
1,1723334400,50448.225454,1.322417e+08,659679.885983,1.688625e+09
2,1723248000,44853.237727,1.171687e+08,662234.901217,1.726121e+09
3,1723161600,72454.880853,1.911409e+08,663002.927410,1.719836e+09
4,1723075200,80170.117144,1.998172e+08,659674.302984,1.771010e+09
5,1722988800,81593.636806,1.999090e+08,670805.268229,1.569934e+09
6,1722902400,74211.307198,1.851430e+08,665384.863037,1.634111e+09
7,1722816000,313459.076891,7.255631e+08,663491.748516,1.605959e+09
8,1722729600,65045.254752,1.831165e+08,647764.560504,1.738812e+09
9,1722643200,55157.496087,1.628263e+08,640436.236770,1.861113e+09
